In [1]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
#import panel as pn
#pn.extension(‘plotly’)
#import plotly.express as px
#import hvplot.pandas
#import matplotlib.pyplot as plt
#import numpy as np

In [2]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [4]:
# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes: 5 years.
start_date = pd.Timestamp("2018-02-9", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-01-28", tz="America/New_York").isoformat()

# Set the ticker information

tickers = ["XLE","XLF","XLRE", "XLK", "XLP", "XLB", "XLI", "XLV", "XLU", "XLY", "XLC"]

# Get 3 year's worth of historical price data for Microsoft and Coca-Cola
df_ticker = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Display sample data
df_ticker.head()

XLB                                  XLC       \
                            open   high    low   close    volume open high   
time                                                                         
2018-02-09 00:00:00-05:00  58.05  58.81  56.62  58.410  24625943  NaN  NaN   
2018-02-12 00:00:00-05:00  58.88  59.95  58.71  59.600   9802061  NaN  NaN   
2018-02-13 00:00:00-05:00  59.36  59.72  59.22  59.455   6546863  NaN  NaN   
2018-02-14 00:00:00-05:00  58.96  60.37  58.84  60.250   7487715  NaN  NaN   
2018-02-15 00:00:00-05:00  60.68  60.72  59.96  60.700   5253495  NaN  NaN   

                                            ...    XLV                         \
                          low close volume  ...   open     high    low  close   
time                                        ...                                 
2018-02-09 00:00:00-05:00 NaN   NaN    NaN  ...  81.69  82.8000  79.56  82.10   
2018-02-12 00:00:00-05:00 NaN   NaN    NaN  ...  82.63  83.5149  81.89  82.86   
2018-02-13 00:00:00-05:00 NaN   NaN    NaN  ...  82.39  83.1150  81.88  82.92   
2018-02-14 00:00:00-05:00 NaN   NaN    NaN  ...  82.40  84.0150  82.32  83.86   
2018-02-15 00:00:00-05:00 NaN   NaN    NaN  ...  84.44  84.8300  83.59  84.83   

                                        XLY                              \
                             volume    open      high       low   close   
time                                                                      
2018-02-09 00:00:00-05:00  31673688  101.14  101.6659   97.1001  100.84   
2018-02-12 00:00:00-05:00  12330849  101.79  102.7650  100.7453  102.21   
2018-02-13 00:00:00-05:00   7730724  101.79  102.9700  101.6200  102.74   
2018-02-14 00:00:00-05:00   9151964  102.24  104.5700  102.1700  104.36   
2018-02-15 00:00:00-05:00   8751008  105.00  105.3000  103.8700  105.29   

                                     
                             volume  
time                                 
2018-02-09 00:00:00-05:00  19138030  
2018-02-12 00:00:00-05:00  11212090  
2018-02-13 00:00:00-05:00   4631160  
2018-02-14 00:00:00-05:00   6136119  
2018-02-15 00:00:00-05:00   5111940  

[5 rows x 55 columns]

In [5]:
# multi-level column deconstruction
close_df = df_ticker.iloc[:, df_ticker.columns.get_level_values(1)=='close']
close_df.columns = close_df.columns.droplevel(1)
close_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
time,,,,,,,,,,,
2018-02-09 00:00:00-05:00,58.410,NaN,66.65,27.69,73.810,63.53,53.71,30.09,48.57,82.10,100.84
2018-02-12 00:00:00-05:00,59.600,NaN,67.77,28.06,74.990,64.67,54.27,30.18,48.94,82.86,102.21
2018-02-13 00:00:00-05:00,59.455,NaN,67.48,28.19,75.170,64.84,54.50,30.39,49.16,82.92,102.74
2018-02-14 00:00:00-05:00,60.250,NaN,68.46,28.85,76.110,66.03,54.49,30.20,48.61,83.86,104.36
2018-02-15 00:00:00-05:00,60.700,NaN,68.28,29.02,77.255,67.27,55.38,30.47,49.70,84.83,105.29
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24 00:00:00-05:00,84.090,72.73,62.86,38.31,101.760,155.11,75.72,46.99,68.18,129.02,182.59
2022-01-25 00:00:00-05:00,83.330,71.27,65.30,38.47,100.780,151.61,74.87,46.86,67.11,128.21,179.47
2022-01-26 00:00:00-05:00,82.430,70.16,65.18,38.57,99.870,152.42,74.28,46.04,66.56,127.54,178.70


In [6]:
# Remove NAs
close_df.dropna(inplace = True)
close_df

/Users/prestonkirschner/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
time,,,,,,,,,,,
2018-06-19 00:00:00-04:00,58.52,49.96,74.73,27.360,73.24,71.40,51.05,31.500,50.22,84.85,111.77
2018-06-20 00:00:00-04:00,58.30,50.58,75.08,27.265,73.26,71.58,51.11,31.850,50.23,85.03,112.30
2018-06-21 00:00:00-04:00,57.70,50.27,73.62,27.215,72.38,71.02,51.20,32.040,50.40,84.56,111.49
2018-06-22 00:00:00-04:00,58.49,50.49,75.21,27.070,72.60,70.80,51.62,32.300,50.75,84.91,111.34
2018-06-25 00:00:00-04:00,57.63,49.45,73.56,26.790,71.70,69.34,51.87,32.235,51.60,84.15,108.91
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24 00:00:00-05:00,84.09,72.73,62.86,38.310,101.76,155.11,75.72,46.990,68.18,129.02,182.59
2022-01-25 00:00:00-05:00,83.33,71.27,65.30,38.470,100.78,151.61,74.87,46.860,67.11,128.21,179.47
2022-01-26 00:00:00-05:00,82.43,70.16,65.18,38.570,99.87,152.42,74.28,46.040,66.56,127.54,178.70


In [7]:
stock_returns = close_df.pct_change()
stock_returns.dropna(inplace = True)
stock_returns

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
time,,,,,,,,,,,
2018-06-20 00:00:00-04:00,-0.003759,0.012410,0.004684,-0.003472,0.000273,0.002521,0.001175,0.011111,0.000199,0.002121,0.004742
2018-06-21 00:00:00-04:00,-0.010292,-0.006129,-0.019446,-0.001834,-0.012012,-0.007823,0.001761,0.005965,0.003384,-0.005527,-0.007213
2018-06-22 00:00:00-04:00,0.013692,0.004376,0.021597,-0.005328,0.003040,-0.003098,0.008203,0.008115,0.006944,0.004139,-0.001345
2018-06-25 00:00:00-04:00,-0.014703,-0.020598,-0.021939,-0.010344,-0.012397,-0.020621,0.004843,-0.002012,0.016749,-0.008951,-0.021825
2018-06-26 00:00:00-04:00,0.003644,0.001662,0.013866,-0.003733,0.003208,0.003750,-0.004049,0.005429,0.000000,-0.002971,0.006978
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24 00:00:00-05:00,0.001191,0.007480,0.006084,0.003142,0.005236,0.004989,-0.002897,0.002133,-0.009875,-0.003322,0.011747
2022-01-25 00:00:00-05:00,-0.009038,-0.020074,0.038816,0.004176,-0.009631,-0.022565,-0.011226,-0.002767,-0.015694,-0.006278,-0.017087
2022-01-26 00:00:00-05:00,-0.010800,-0.015575,-0.001838,0.002599,-0.009030,0.005343,-0.007880,-0.017499,-0.008195,-0.005226,-0.004290


In [14]:
stock_returns['date'] = stock_returns.index.date
stock_returns

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,date
time,,,,,,,,,,,,
2018-06-20 00:00:00-04:00,-0.003759,0.012410,0.004684,-0.003472,0.000273,0.002521,0.001175,0.011111,0.000199,0.002121,0.004742,2018-06-20
2018-06-21 00:00:00-04:00,-0.010292,-0.006129,-0.019446,-0.001834,-0.012012,-0.007823,0.001761,0.005965,0.003384,-0.005527,-0.007213,2018-06-21
2018-06-22 00:00:00-04:00,0.013692,0.004376,0.021597,-0.005328,0.003040,-0.003098,0.008203,0.008115,0.006944,0.004139,-0.001345,2018-06-22
2018-06-25 00:00:00-04:00,-0.014703,-0.020598,-0.021939,-0.010344,-0.012397,-0.020621,0.004843,-0.002012,0.016749,-0.008951,-0.021825,2018-06-25
2018-06-26 00:00:00-04:00,0.003644,0.001662,0.013866,-0.003733,0.003208,0.003750,-0.004049,0.005429,0.000000,-0.002971,0.006978,2018-06-26
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24 00:00:00-05:00,0.001191,0.007480,0.006084,0.003142,0.005236,0.004989,-0.002897,0.002133,-0.009875,-0.003322,0.011747,2022-01-24
2022-01-25 00:00:00-05:00,-0.009038,-0.020074,0.038816,0.004176,-0.009631,-0.022565,-0.011226,-0.002767,-0.015694,-0.006278,-0.017087,2022-01-25
2022-01-26 00:00:00-05:00,-0.010800,-0.015575,-0.001838,0.002599,-0.009030,0.005343,-0.007880,-0.017499,-0.008195,-0.005226,-0.004290,2022-01-26
